In [ ]:
import time
import sys
import speech_recognition as sr
import pyttsx3

In [ ]:
LANGUAGE_CODE = "en-IN"
LISTEN_TIMEOUT = 5
PHRASE_TIME_LIMIT = 6
AMBIENT_NOISE_DURATION = 0.1
TTS_RATE = 150
TTS_VOLUME = 1.0

In [ ]:
recognizer = sr.Recognizer()

In [ ]:
def speak(text: str):
    if not text:
        return

    try:
        engine = pyttsx3.init(driverName='sapi5')
        engine.setProperty("rate", TTS_RATE)
        engine.setProperty("volume", TTS_VOLUME)

        engine.say(text)
        print(text, "\n")
        engine.runAndWait()
        engine.stop()

        time.sleep(0.5)

    except Exception as e:
        print(f"[TTS Error] {e}")


In [ ]:
def listen_once() -> str | None:
    """
    Captures audio from default microphone and return recognized text.
    """
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source, AMBIENT_NOISE_DURATION)
        print("[Listening] Speak now...")
        try:
            audio = recognizer.listen(source, timeout=LISTEN_TIMEOUT, phrase_time_limit=PHRASE_TIME_LIMIT)
        except sr.WaitTimeoutError:
            print("No speech detected (timeout).")
            return None

    try:
        text = recognizer.recognize_google(audio, language=LANGUAGE_CODE)
        print(text)
        return text
    except sr.UnknownValueError:
        print("[Error] Speech not recognized (UnknownValueError).")
        return None
    except sr.RequestError as e:
        print(f"API / network error: {e}")
        return "__API_ERROR__"


In [ ]:
def handle_command(cmd_text: str) -> bool:
    """
    Recognizes a small set of commands and replies by voice.
    """
    if not cmd_text:
        speak("Sorry, I didn't catch that. Please speak again.")
        return True

    cmd = cmd_text.strip().lower()

    if any(word in cmd for word in ("hello", "hi", "hey")):
        speak("Heyy. Try 'check inbox', 'send email', 'read email', or 'exit'.")
        return True

    if "check inbox" in cmd or "check my inbox" in cmd or ("check" in cmd and "inbox" in cmd):
        speak("Inbox feature will be added in the next milestone.")
        return True

    if "read email" in cmd or "read my email" in cmd:
        speak("Read email feature will be added in the next milestone.")
        return True

    if "send email" in cmd or "compose email" in cmd:
        speak("Sending email is a future feature.")
        return True

    if "exit" in cmd or "quit" in cmd or "stop" in cmd:
        speak("Exiting demo.. Byeee.")
        return False

    speak("Command not recognised")
    return True

In [ ]:
def interactive_loop():
    """Main voice interaction loop."""
    speak("Voice demo started. ")
    running = True
    while running:
        text = listen_once()
        if text == "__API_ERROR__":
            speak("Network error when contacting speech service. Check your internet connection.")
            # wait a moment to avoid busy-looping
            time.sleep(1.0)
            continue
        running = handle_command(text)
        # small pause before next listen to avoid overlap
        time.sleep(1)

In [8]:
if __name__ == "__main__":
    try:
        interactive_loop()
    except KeyboardInterrupt:
        print("\n[Interrupted]")
        speak("Demo interrupted. Goodbye.")
        sys.exit(0)